In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('../data/tokyo.xlsx')
data.drop('MunicipalityCode', axis=1, inplace=True)
data.drop('DistrictCode', axis=1, inplace=True)

data = data.rename(columns={
    'TradePrice': 'TradePriceYen',
    'Direction': 'RoadDirection'
})

In [3]:
# original Japanese names of Wards/Cities

municipality_mapping = {
    'Chiyoda Ward': '千代田区 (Chiyoda Ward)',
    'Chuo Ward': '中央区 (Chuo Ward)',
    'Minato Ward': '港区 (Minato Ward)',
    'Shinjuku Ward': '新宿区 (Shinjuku Ward)',
    'Bunkyo Ward': '文京区 (Bunkyo Ward)',
    'Taito Ward': '台東区 (Taito Ward)',
    'Sumida Ward': '墨田区 (Sumida Ward)',
    'Koto Ward': '江東区 (Koto Ward)',
    'Shinagawa Ward': '品川区 (Shinagawa Ward)',
    'Meguro Ward': '目黒区 (Meguro Ward)',
    'Ota Ward': '大田区 (Ota Ward)',
    'Setagaya Ward': '世田谷区 (Setagaya Ward)',
    'Shibuya Ward': '渋谷区 (Shibuya Ward)',
    'Nakano Ward': '中野区 (Nakano Ward)',
    'Suginami Ward': '杉並区 (Suginami Ward)',
    'Toshima Ward': '豊島区 (Toshima Ward)',
    'Kita Ward': '北区 (Kita Ward)',
    'Arakawa Ward': '荒川区 (Arakawa Ward)',
    'Itabashi Ward': '板橋区 (Itabashi Ward)',
    'Nerima Ward': '練馬区 (Nerima Ward)',
    'Adachi Ward': '足立区 (Adachi Ward)',
    'Katsushika Ward': '葛飾区 (Katsushika Ward)',
    'Edogawa Ward': '江戸川区 (Edogawa Ward)',
    'Hachioji City': '八王子市 (Hachioji City)',
    'Tachikawa City': '立川市 (Tachikawa City)',
    'Musashino City': '武蔵野市 (Musashino City)',
    'Mitaka City': '三鷹市 (Mitaka City)',
    'Oume City': '青梅市 (Oume City)',
    'Fuchu City': '府中市 (Fuchu City)',
    'Akishima City': '昭島市 (Akishima City)',
    'Chofu City': '調布市 (Chofu City)',
    'Machida City': '町田市 (Machida City)',
    'Koganei City': '小金井市 (Koganei City)',
    'Kodaira City': '小平市 (Kodaira City)',
    'Hino City': '日野市 (Hino City)',
    'Higashimurayama City': '東村山市 (Higashimurayama City)',
    'Kokubunji City': '国分寺市 (Kokubunji City)',
    'Kunitachi City': '国立市 (Kunitachi City)',
    'Fussa City': '福生市 (Fussa City)',
    'Komae City': '狛江市 (Komae City)',
    'Higashiyamato City': '東大和市 (Higashiyamato City)',
    'Kiyose City': '清瀬市 (Kiyose City)',
    'Higashikurume City': '東久留米市 (Higashikurume City)',
    'Musashimurayama City': '武蔵村山市 (Musashimurayama City)',
    'Tama City': '多摩市 (Tama City)',
    'Inagi City': '稲城市 (Inagi City)',
    'Hamura City': '羽村市 (Hamura City)',
    'Akiruno City': 'あきる野市 (Akiruno City)',
    'Nishitokyo City': '西東京市 (Nishitokyo City)',
    'Mizuho Town, Nishitama County': '瑞穂町 (Mizuho Town, Nishitama County)',
    'Hinode Town, Nishitama County': '日の出町 (Hinode Town, Nishitama County)',
    'Hinohara Village, Nishitama County': '檜原村 (Hinohara Village, Nishitama County)',
    'Okutama Town, Nishitama County': '奥多摩町 (Okutama Town, Nishitama County)',
    'Oshima Town': '大島町 (Oshima Town)',
    'Niijima Village': '新島村 (Niijima Village)',
    'Miyake Village': '三宅村 (Miyake Village)',
    'Hachijo Town': '八丈町 (Hachijo Town)',
    'Ogasawara Village': '小笠原村 (Ogasawara Village)',
    'Kozushima Village': '神津島村 (Kozushima Village)',
}

In [4]:
# add original Japanese

data['Municipality'] = data['Municipality'].map(municipality_mapping)
data['Prefecture'] = '東京都 (Tokyo)'

In [5]:
# enforce integer types on integer columns which contain nan values

data.PricePerUnit = data.PricePerUnit.astype('Int64')
data.UnitPrice = data.UnitPrice.astype('Int64')
data.TotalFloorArea = data.TotalFloorArea.astype('Int64')
data.CoverageRatio = data.CoverageRatio.astype('Int64')
data.FloorAreaRatio = data.FloorAreaRatio.astype('Int64')

In [6]:
# flag columns which have floored/capped values

# BuildingYear has a value 'before the war' which acts as a floor year
# we will use the 'flag and floor method' before training the XGBoost model:
# set the value as 1945 but flag it as floored in a separate column
data['BuildingYearFloored'] = data.BuildingYear.apply(lambda x: 1 if x == 'before the war' else 0)
data.BuildingYear = data.BuildingYear.apply(lambda x: 1945 if x == 'before the war' else x).astype(float).astype('Int64')

# Frontage has a value 9999.9 that appears for capped values
# we will leave the value alone but flag it for XGBoost
data['FrontageCapped'] = data.Frontage.apply(lambda x: 1 if x == 9999.9 else 0)

# TotalFloorArea has a value 9999 that appears for capped values
# we will leave the value alone but flag it for XGBoost
data['TotalFloorAreaCapped'] = data.TotalFloorArea.apply(lambda x: 1 if x == 9999 else 0)

In [7]:
# Period has string format eg '2nd quarter 2010'

data['TransactionYear'] = data.Period.apply(lambda x: int(x[-4:]))
data['TransactionQuarter'] = data.Period.apply(lambda x: int(x[0]))

# create a separate a datetime column to replace Period, using the end date of each quarter
quarter_end = {
    1: (3, 31),
    2: (6, 30),
    3: (9, 30),
    4: (12, 31)
}

data['TransactionQuarterEndDate'] = data.apply(
    lambda row: pd.Timestamp(row['TransactionYear'],
                             quarter_end[row['TransactionQuarter']][0],
                             quarter_end[row['TransactionQuarter']][1]), axis=1)

data.drop('Period', axis=1, inplace=True)

In [8]:
print('columns:', len(data.columns))
print('rows:', len(data))

columns: 32
rows: 607756


In [9]:
data.head()

,PriceCategory,Type,Region,Prefecture,Municipality,DistrictName,TradePriceYen,PricePerUnit,FloorPlan,Area,...,CoverageRatio,FloorAreaRatio,Renovation,Remarks,BuildingYearFloored,FrontageCapped,TotalFloorAreaCapped,TransactionYear,TransactionQuarter,TransactionQuarterEndDate
0,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,東京都 (Tokyo),千代田区 (Chiyoda Ward),Higashikanda,14000000,<NA>,NaN,120,...,80,500,NaN,Dealings including private road,0,0,0,2010,2,2010-06-30
1,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,東京都 (Tokyo),千代田区 (Chiyoda Ward),Higashikanda,7300000,<NA>,1K,20,...,80,600,Not yet,NaN,0,0,0,2010,2,2010-06-30
2,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,東京都 (Tokyo),千代田区 (Chiyoda Ward),Higashikanda,8200000,<NA>,1K,20,...,80,700,Not yet,NaN,0,0,0,2010,2,2010-06-30
3,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,東京都 (Tokyo),千代田区 (Chiyoda Ward),Higashikanda,10000000,<NA>,NaN,20,...,80,500,NaN,NaN,0,0,0,2010,2,2010-06-30
4,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,東京都 (Tokyo),千代田区 (Chiyoda Ward),Iwamotocho,11000000,<NA>,1R,20,...,80,700,Not yet,NaN,0,0,0,2010,2,2010-06-30


In [10]:
data.to_excel('../data/tokyo-clean.xlsx', index=False)